In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
os.environ['PATH_TO_ENV'] = "~/projects/chatsky-llm-autoconfig/.env"
os.getenv('EMBEDDER_MODEL')

'BAAI/bge-m3'

In [ ]:
from dialogue2graph.pipelines.core.dialogue import DialogueMessage, Dialogue
import pickle
import numpy as np

/home/olshevskaya/miniconda3/envs/conda_env_py311/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [185]:
with open("gen_dataset_augment_naive", "rb") as fp:
    data = pickle.load(fp)

In [186]:
len(data)

160

In [77]:
from sentence_transformers import SentenceTransformer
from dialogue2graph.metrics.no_llm_metrics.metrics import match_roles, is_correct_length
from functools import reduce
from tqdm import tqdm

### Checking duplicates within dialogues per graph

In [143]:
def check_no_duplicates_all_dialogues(dialogues):
    all_utterances = []
    for dia in dialogues:
        utterances = [uttr['text'] for uttr in dia]
        all_utterances.append(utterances)

    # Функция для преобразования списка в множество
    to_set = lambda x: set(x)
    # Преобразование каждого списка в множество
    sets = map(to_set, all_utterances)
    # Нахождение общего пересечения всех множеств
    common_elements = reduce(lambda a, b: a & b, sets)
    if len(common_elements) == 0:
        return True
    else:
        print('common_elements:', common_elements)
        return False

In [187]:
errors = []
failure_examples = []

for i, example in enumerate(tqdm(data)):
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]

    try:
        if not check_no_duplicates_all_dialogues(augmented_dialogues):
            failure_examples.append(i)
    except Exception as e:
        errors.append((i, e))

100%|██████████| 160/160 [00:00<00:00, 16918.06it/s]

common_elements: {'Hi there! How can I help you today?'}
common_elements: {'Hi there! How can I help you today?'}
common_elements: {'Hi there! How can I help you today?', 'In the morning.'}
common_elements: {'Hi there! How can I help you today?'}
common_elements: {"My device isn't being recognized."}
common_elements: {'Hi there! How can I help you today?', 'My account number is 123456.'}
common_elements: {'Hi there! How can I help you today?'}
common_elements: {'123456'}
common_elements: {'Hi there! How can I help you today?'}
common_elements: {'Hi there! How can I help you today?'}
common_elements: {'Hi there! How can I help you today?', "It's on the installation page, specifically in the prerequisites section."}
common_elements: {'Hi there! How can I help you today?'}
common_elements: {'Hi there! How can I help you today?'}
common_elements: {'Hi there! How can I help you today?'}
common_elements: {'Hi there! How can I help you today?'}
common_elements: {'Hi there! How can I help you 

In [188]:
# duplicates
print(failure_examples)

[8, 12, 19, 25, 26, 34, 35, 37, 39, 41, 46, 54, 58, 59, 60, 61, 65, 69, 71, 72, 81, 82, 87, 93, 101, 103, 104, 105, 107, 108, 110, 111, 119, 120, 148]


In [189]:
# generation errors
errors

[(6, TypeError("'NoneType' object is not iterable")),
 (30, TypeError("'NoneType' object is not iterable")),
 (66, TypeError("'NoneType' object is not iterable")),
 (74, TypeError("'NoneType' object is not iterable"))]

### Checking duplicates within one dialogue

In [190]:
def check_no_duplicates_one_dialogue(dialogue):
    utterances = [uttr['text'] for uttr in dialogue]

    if len(utterances) == len(set(utterances)):
        return True
        
    else:
        uttr_count = {}
        for uttr in utterances:
            uttr_count.setdefault(uttr, 0)
            uttr_count[uttr] += 1
            common_elements = [k for k, v in uttr_count.items() if v > 1]
        print('common_elements:', common_elements)
        return False

In [191]:
errors = []
failure_examples = []

for i, example in enumerate(tqdm(data)):
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]

    for j, aug_dia in enumerate(augmented_dialogues):
        try:
            if check_no_duplicates_one_dialogue(aug_dia):
                continue
            else:
                failure_examples.append((i, j))
        except Exception as e:
            errors.append((i, j, e))

100%|██████████| 160/160 [00:00<00:00, 17439.93it/s]

common_elements: ['Red, please.']
common_elements: ['Hi there! How can I help you today?']
common_elements: ['Hi there! How can I help you today?']
common_elements: ['Hi there! How can I help you today?']
common_elements: ['I see that you have some concerns regarding our ethical sourcing practices. Can you share more details about what specifically troubles you?']
common_elements: ['I see that you have concerns regarding our ethical sourcing. Can you specify what aspect is troubling you?']
common_elements: ['I see that you have some concerns regarding ethical sourcing. Can you tell me more about what specifically troubles you?']
common_elements: ['Grammar']
common_elements: ['1.5 BTC']
common_elements: ['1.5 BTC']
common_elements: ['1.5 BTC']
common_elements: ["Hi there! I'm here to address any inquiries or worries you might have regarding our drone delivery service. What can I help you with today?"]
common_elements: ["Hi there! I'm here to address any inquiries or worries you might ha

In [177]:
print(len(failure_examples))
print(failure_examples)

74
[(0, 0), (2, 5), (2, 6), (2, 7), (4, 2), (4, 3), (4, 9), (7, 14), (12, 0), (12, 1), (12, 2), (17, 12), (17, 13), (17, 14), (17, 16), (17, 17), (18, 6), (19, 0), (19, 1), (19, 3), (19, 4), (26, 3), (28, 5), (34, 2), (34, 3), (37, 0), (37, 1), (37, 2), (37, 3), (37, 4), (43, 1), (44, 4), (54, 3), (54, 7), (63, 5), (78, 9), (78, 10), (78, 13), (83, 0), (87, 0), (90, 0), (90, 1), (90, 2), (90, 3), (90, 4), (90, 5), (95, 6), (97, 2), (97, 3), (98, 0), (98, 1), (98, 2), (98, 3), (98, 4), (98, 5), (98, 6), (98, 7), (99, 0), (99, 1), (99, 2), (99, 3), (99, 4), (99, 5), (101, 1), (109, 6), (114, 2), (116, 0), (116, 3), (116, 4), (118, 0), (118, 1), (118, 2), (118, 3), (118, 4)]


In [192]:
errors

[(6, 3, TypeError("'NoneType' object is not iterable")),
 (30, 0, TypeError("'NoneType' object is not iterable")),
 (66, 1, TypeError("'NoneType' object is not iterable")),
 (74, 9, TypeError("'NoneType' object is not iterable"))]

### Checking matching roles and length correctness

In [193]:
errors = []
failure_examples = []

for i, example in enumerate(tqdm(data)):
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]

    try:
        for j, (orig_dia, aug_dia) in enumerate(zip(dialogues, augmented_dialogues)):
            if match_roles(Dialogue.from_list(orig_dia), Dialogue.from_list(aug_dia)):
                continue
            else:
                failure_examples.append((i, j))
    except Exception as e:
        errors.append((i, j, e))

  0%|          | 0/160 [00:00<?, ?it/s]

100%|██████████| 160/160 [00:00<00:00, 2064.87it/s]


In [194]:
print(len(failure_examples))
print(failure_examples)

0
[]


In [195]:
errors

[(6, 3, TypeError("'NoneType' object is not iterable")),
 (7,
  0,
  1 validation error for DialogueMessage
  participant
    Field required [type=missing, input_value={'text': 'Hi'}, input_type=dict]
      For further information visit https://errors.pydantic.dev/2.10/v/missing),
 (30, 0, TypeError("'NoneType' object is not iterable")),
 (66, 1, TypeError("'NoneType' object is not iterable")),
 (74, 9, TypeError("'NoneType' object is not iterable")),
 (122,
  5,
  1 validation error for DialogueMessage
  participant
    Field required [type=missing, input_value={'text': "I'm sorry I cou...esitate to contact us."}, input_type=dict]
      For further information visit https://errors.pydantic.dev/2.10/v/missing)]

In [196]:
errors = []
failure_examples = []

for i, example in enumerate(tqdm(data)):
    dialogues = [dial['messages'] for dial in example['dialogues']]
    augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]

    try:
        for j, (orig_dia, aug_dia) in enumerate(zip(dialogues, augmented_dialogues)):
            if is_correct_length(Dialogue.from_list(orig_dia), Dialogue.from_list(aug_dia)):
                continue
            else:
                failure_examples.append((i, j))
    except Exception as e:
        errors.append((i, j, e))

100%|██████████| 160/160 [00:00<00:00, 2220.19it/s]


In [197]:
print(len(failure_examples))
print(failure_examples)

4
[(27, 2), (54, 1), (92, 0), (92, 6)]


In [198]:
errors

[(6, 3, TypeError("'NoneType' object is not iterable")),
 (7,
  0,
  1 validation error for DialogueMessage
  participant
    Field required [type=missing, input_value={'text': 'Hi'}, input_type=dict]
      For further information visit https://errors.pydantic.dev/2.10/v/missing),
 (30, 0, TypeError("'NoneType' object is not iterable")),
 (66, 1, TypeError("'NoneType' object is not iterable")),
 (74, 9, TypeError("'NoneType' object is not iterable")),
 (122,
  5,
  1 validation error for DialogueMessage
  participant
    Field required [type=missing, input_value={'text': "I'm sorry I cou...esitate to contact us."}, input_type=dict]
      For further information visit https://errors.pydantic.dev/2.10/v/missing)]

### check_diagonal_similarity

In [ ]:
def check_diagonal_similarity(dialogue_1, dialogue_2, embedder=os.getenv('EMBEDDER_MODEL')):
    utterances_1 = [uttr['text'] for uttr in dialogue_1]
    utterances_2 = [uttr['text'] for uttr in dialogue_2]

    model = SentenceTransformer(embedder)
    embeddings_1 = model.encode(utterances_1)
    embeddings_2 = model.encode(utterances_2)

    model.similarity_fn_name = 'cosine'    
    similarities = model.similarity(embeddings_1, embeddings_2)

    diagonal_elements = np.diag(similarities)
    non_diagonal_elements = similarities[np.where(~np.eye(len(similarities), dtype=bool))]
    
    for i in range(len(diagonal_elements)):
        if diagonal_elements[i] <= non_diagonal_elements.max():
            print('mean aug similarity:', np.mean(diagonal_elements))
            return False
        
    print('mean aug similarity:', np.mean(diagonal_elements))
    return True

In [199]:
example = data[2]
dialogues = [dial['messages'] for dial in example['dialogues']]
augmented_dialogues = [dial['messages'] for dial in example['augmented_dialogues']]
check_no_duplicates_all_dialogues(augmented_dialogues)

True

Изначально некорректный диалог - 'Hello! How can I assist you today?'/'Hi there! How can I help you today?' в середине диалога.

Влияет на косинусное сходство.

In [202]:
dialogues[5]

[{'participant': 'assistant', 'text': 'Hello! How can I assist you today?'},
 {'participant': 'user',
  'text': 'I want to report a fake social media account.'},
 {'participant': 'assistant',
  'text': "I'm sorry to hear that. Can you provide more details about the fake account?"},
 {'participant': 'user', 'text': 'Actually, I want to manage something else.'},
 {'participant': 'assistant', 'text': 'Sure! How else may I help you today?'},
 {'participant': 'user', 'text': 'I have another question.'},
 {'participant': 'assistant', 'text': 'Hello! How can I assist you today?'},
 {'participant': 'user',
  'text': 'I need assistance managing my review responses on Google/Yelp.'},
 {'participant': 'assistant',
  'text': 'Certainly! Are you looking to respond to positive or negative reviews?'},
 {'participant': 'user', 'text': 'Positive reviews.'},
 {'participant': 'assistant',
  'text': 'Acknowledge positive feedback and thank customers for their support to encourage loyalty.'},
 {'participan

In [ ]:
augmented_dialogues[5]

[{'text': 'Hi there! How can I help you today?', 'participant': 'assistant'},
 {'text': 'I need to report a fake account on social media.',
  'participant': 'user'},
 {'text': "I'm really sorry to hear that. Could you share some details about the account in question?",
  'participant': 'assistant'},
 {'text': "Actually, I've got a different issue to address.",
  'participant': 'user'},
 {'text': 'Of course! What else can I assist you with today?',
  'participant': 'assistant'},
 {'text': 'I have another inquiry.', 'participant': 'user'},
 {'text': 'Hi there! How can I help you today?', 'participant': 'assistant'},
 {'text': 'I need help managing my responses to reviews on Google/Yelp.',
  'participant': 'user'},
 {'text': 'Absolutely! Are you focusing on responding to positive reviews or negative ones?',
  'participant': 'assistant'},
 {'text': 'Positive feedback.', 'participant': 'user'},
 {'text': 'Make sure to express gratitude for the positive comments to strengthen customer loyalt

In [169]:
for orig_dia, aug_dia in zip(dialogues, augmented_dialogues):
    print(check_diagonal_similarity(orig_dia, aug_dia))

mean aug similarity: 0.95032436
False
mean aug similarity: 0.9559381
False
mean aug similarity: 0.9606403
False
mean aug similarity: 0.9370233
False
mean aug similarity: 0.9388061
False
